In [ ]:
import torch
import detectron2
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

# Step 1: Register the dataset
register_coco_instances("custom_dataset", {}, "path_to_train_annotation_json", "path_to_train_images_folder")

# Step 2: Configure the model
cfg = get_cfg()
cfg.merge_from_file("path_to_config_file.yaml")
cfg.DATASETS.TRAIN = ("custom_dataset",)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = "path_to_pretrained_weights"
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes

# Step 3: Define a custom trainer
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

# Step 4: Train the model
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Step 5: Evaluate the model
evaluator = COCOEvaluator("custom_dataset", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "custom_dataset")
inference_on_dataset(trainer.model, val_loader, evaluator)

# Step 6: Perform inference on new images
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

# Step 7: Visualize the results
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow("Segmentation Result", out.get_image()[:, :, ::-1])
cv2.waitKey(0)
